# Oracle database load profile analyse 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

## LOAD DATASETS

In [3]:
df_fats = pd.read_csv('https://raw.githubusercontent.com/naumovskiy/HW4/main/HW4/oracle_data/fat_sql_17122020.dat',
                      header=0, sep='\t',parse_dates=['BEGIN_INTERVAL_TIME'])
df_awr = pd.read_csv('https://raw.githubusercontent.com/naumovskiy/HW4/main/HW4/oracle_data/awr_17122020.dat',
                     header=0, sep='\t',parse_dates=['TT'])

### First analyse SQL_ID data

In [4]:
df_fats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21248 entries, 0 to 21247
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   BEGIN_INTERVAL_TIME  21248 non-null  datetime64[ns]
 1   SQL_ID               21248 non-null  object        
 2   ELA                  21248 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 498.1+ KB


In [5]:
df_fats.describe()

,ELA
count,21248.000000
mean,113.346242
std,1796.436489
min,0.000000
25%,0.027800
50%,0.043800
75%,0.144400
max,50939.367000


In [6]:
df_fats.head(2)

,BEGIN_INTERVAL_TIME,SQL_ID,ELA
0,2020-12-17 07:00:53.109,ff30u4fk8c21j,0.0993
1,2020-12-17 07:00:53.109,2yr5hb3u7uvmq,0.0171


### First analyse AWR data

In [7]:
df_awr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31721 entries, 0 to 31720
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   TT                  31721 non-null  datetime64[ns]
 1   WAIT_CLASS          31721 non-null  object        
 2   EVENT_              31721 non-null  object        
 3   SQL_OPNAME          31721 non-null  object        
 4   SQL_PLAN_OPERATION  31721 non-null  object        
 5   SQL_PLAN_OPTIONS    31721 non-null  object        
 6   SQL_ID              30522 non-null  object        
 7   CNT                 31721 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.9+ MB


In [8]:
df_awr.describe()

,CNT
count,31721.000000
mean,5.163362
std,23.763666
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,496.000000


In [10]:
df_awr.head(3)

,TT,WAIT_CLASS,EVENT_,SQL_OPNAME,SQL_PLAN_OPERATION,SQL_PLAN_OPTIONS,SQL_ID,CNT
0,2020-12-17 07:00:00,ONCPU,ONCPU,UPDATE,UPDATE,EMPTY,aw3kh5r15pnmq,1
1,2020-12-17 07:00:00,ONCPU,ONCPU,SELECT,COUNT,STOPKEY,9ac7scq121506,1
2,2020-12-17 07:00:00,ONCPU,ONCPU,DELETE,DELETE STATEMENT,EMPTY,cg1j28mh9sn83,1


# PREPARE DATA

## FATS SQL_ID

In [11]:
df_fats.head(2)

,BEGIN_INTERVAL_TIME,SQL_ID,ELA
0,2020-12-17 07:00:53.109,ff30u4fk8c21j,0.0993
1,2020-12-17 07:00:53.109,2yr5hb3u7uvmq,0.0171


#### Unreadable graph. Too much SQL_ID

In [13]:
fig = px.line(df_fats, x="BEGIN_INTERVAL_TIME", y='ELA',color='SQL_ID',hover_name='SQL_ID',log_y=True,
        width=900,height=400,title='ALL SQL_ID gererate by period',
        labels=dict(BEGIN_INTERVAL_TIME="current snap start time",ELA="elapsed time (sec.)"))
fig.update_layout(margin=dict(l=10, r=10, t=50, b=0))
fig.show()

## AWR DATA

In [15]:
df_awr.head(3)

,TT,WAIT_CLASS,EVENT_,SQL_OPNAME,SQL_PLAN_OPERATION,SQL_PLAN_OPTIONS,SQL_ID,CNT
0,2020-12-17 07:00:00,ONCPU,ONCPU,UPDATE,UPDATE,EMPTY,aw3kh5r15pnmq,1
1,2020-12-17 07:00:00,ONCPU,ONCPU,SELECT,COUNT,STOPKEY,9ac7scq121506,1
2,2020-12-17 07:00:00,ONCPU,ONCPU,DELETE,DELETE STATEMENT,EMPTY,cg1j28mh9sn83,1


### WAIT CLASS

In [16]:
df_awr_simple =  df_awr.groupby(['TT','WAIT_CLASS']).CNT.agg(['sum']).add_prefix('CNT_').reset_index()
df_awr_simple.head(3)

,TT,WAIT_CLASS,CNT_sum
0,2020-12-17 07:00:00,Application,3
1,2020-12-17 07:00:00,Commit,13
2,2020-12-17 07:00:00,Configuration,1


In [17]:
df_awr_simple['SNAP_ID'] = df_awr_simple.TT.dt.hour + df_awr_simple.TT.dt.minute%15

In [18]:
colors_wait_class = {'Application': 'darkred', 'ONCPU': 'green', 'User I/O': 'blue', 'Commit':'red','Other':'magenta',
        'Network':'darkgreen','System I/O':'darkblue','Configuration':'chocolate','Concurrency':'crimson'}

In [19]:
fig = px.area(df_awr_simple, x="TT", y='CNT_sum',color='WAIT_CLASS',hover_name='WAIT_CLASS',log_y=True,
        color_discrete_map=colors_wait_class, title='WAIT CLASS profile',
        labels=dict(TT='date time from ASH history', CNT_sum='Count event by minute'), width=1100,height=400)
fig.update_layout(margin=dict(l=10, r=10, t=50, b=0))
fig.show()

In [20]:
color_wc_sequence = ['darkred','red','chocolate','green','magenta','darkblue','blue','crimson','darkgreen']

In [21]:
fig=px.pie(df_awr_simple, names="WAIT_CLASS", values='CNT_sum',color='WAIT_CLASS',labels='WAIT_CLASS',width=400,height=400,
       color_discrete_sequence=color_wc_sequence,title='WAIT CLASS distribution',hover_data=['WAIT_CLASS'])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(margin=dict(l=10, r=10, t=50, b=0))
fig.show()

### EVENTS

In [22]:
df_awr_ev_simple =  df_awr.groupby(['TT','WAIT_CLASS','EVENT_']).CNT.agg(['sum']).add_prefix('CNT_').reset_index()
df_awr_ev_simple.head(3)

,TT,WAIT_CLASS,EVENT_,CNT_sum
0,2020-12-17 07:00:00,Application,enq: TX - row lock contention,3
1,2020-12-17 07:00:00,Commit,log file sync,13
2,2020-12-17 07:00:00,Configuration,enq: SQ - contention,1


In [23]:
df_awr_ev_simple['SNAP_ID'] = df_awr_ev_simple.TT.dt.hour + df_awr_ev_simple.TT.dt.minute%15

In [24]:
fig = px.area(df_awr_ev_simple, x="TT", y='CNT_sum',color='EVENT_',hover_name='WAIT_CLASS',log_y=True,
        title='EVETNS profile', width=1100, height=400, labels=dict(TT='date time from ASH history', CNT_sum='Count event by minute'))
fig.update_layout(margin=dict(l=10, r=10, t=50, b=0))
fig.show()

In [25]:
cat = ['Application',	'Commit',	'Concurrency',	'Configuration',	'Network',	'ONCPU',	'Other',	'System I/O',	'User I/O']
fig = px.area(df_awr_ev_simple[df_awr_ev_simple.WAIT_CLASS != 'ONCPU' ], x="TT", y='CNT_sum',color='EVENT_',
        facet_col="WAIT_CLASS",facet_col_wrap=2 ,category_orders={"WAIT_CLASS":cat},width=1100,height=600,
        title='EVENTS distribution by WAIT_CLASS(without CPU)', log_y=True, labels=dict(CNT_sum="",TT=""))
fig.update_layout(margin=dict(l=0, r=0, t=50, b=0))
fig.show()

### SQL_OPNAME

In [26]:
df_awr_sql_simple =  df_awr.groupby(['TT','WAIT_CLASS','SQL_OPNAME']).CNT.agg(['sum']).add_prefix('CNT_').reset_index()
df_awr_sql_simple.head(3)

,TT,WAIT_CLASS,SQL_OPNAME,CNT_sum
0,2020-12-17 07:00:00,Application,SELECT,3
1,2020-12-17 07:00:00,Commit,EMPTY,13
2,2020-12-17 07:00:00,Configuration,SELECT,1


In [27]:
fig = px.bar(df_awr_sql_simple, x="TT", y='CNT_sum',color='SQL_OPNAME',title='SQL OPNAME profile',
        width=1100,height=400, labels=dict(TT='date time from ASH history',CNT_sum='Count SQL OPERATION by minute'))
fig.update_layout(margin=dict(l=10, r=10, t=50, b=0))
fig.show()

In [28]:
fig=px.pie(df_awr_sql_simple, names="SQL_OPNAME", values='CNT_sum',color='SQL_OPNAME',hover_data=['SQL_OPNAME'],
       labels='SQL_OPNAME',width=400,height=400,title='SQL OPNAME distribution')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(margin=dict(l=10, r=10, t=50, b=0))
fig.show()

### SQL PLAN OPER

In [29]:
df_awr['SQL_PLAN_OPTIONS'] = df_awr['SQL_PLAN_OPTIONS'].apply(lambda x: '' if x=='EMPTY' else x)

In [30]:
df_awr['SQL_PLAN'] = df_awr['SQL_PLAN_OPERATION'] +' '+df_awr['SQL_PLAN_OPTIONS']

In [31]:
df_awr_sql_plan_simple =  df_awr.groupby(['TT','WAIT_CLASS','SQL_PLAN','SQL_PLAN_OPERATION',
                                          'SQL_PLAN_OPTIONS']).CNT.agg(['sum']).add_prefix('CNT_').reset_index()
df_awr_sql_plan_simple.head(3)

,TT,WAIT_CLASS,SQL_PLAN,SQL_PLAN_OPERATION,SQL_PLAN_OPTIONS,CNT_sum
0,2020-12-17 07:00:00,Application,FOR UPDATE,FOR UPDATE,,3
1,2020-12-17 07:00:00,Commit,EMPTY,EMPTY,,13
2,2020-12-17 07:00:00,Configuration,SEQUENCE,SEQUENCE,,1


In [32]:
fig = px.area(df_awr_sql_plan_simple, x="TT", y='CNT_sum',color='SQL_PLAN',hover_name='SQL_PLAN_OPERATION',title='SQL PLAN profile',
        labels=dict(TT='date time from ASH history',CNT_sum='Count PLAN OPER by minute'), width=1100,height=400)
fig.update_layout(margin=dict(l=10, r=10, t=50, b=0))
fig.show()

# FIND FATs SQL_ID

In [33]:
def compare_columns(x):
    return sum(x[-3:]) / sum(x[:3]) 

In [35]:
df_fats_group = df_fats.groupby(['SQL_ID','BEGIN_INTERVAL_TIME']).ELA.mean().unstack().add_prefix('TT_')
df_fats_group = df_fats_group.dropna(axis=0)

In [36]:
df_fats_group['diff3'] = df_fats_group.apply(lambda x: compare_columns(x), axis=1)

In [37]:
df_fats_simple = df_fats_group[df_fats_group.diff3 > 1.1]
df_fats_simple.shape[1]

8

In [38]:
sql_id_fat = df_fats_simple.drop(columns=['diff3']).T.columns
df_temp = df_fats[df_fats.SQL_ID.isin(sql_id_fat)].groupby(['BEGIN_INTERVAL_TIME','SQL_ID']).ELA.agg(['mean']).reset_index()

cols = 3
rows = int(np.ceil(float(len(sql_id_fat)/cols)))
fig = make_subplots(rows=rows, cols=cols, subplot_titles=tuple(df_fats_simple.index))
for i, name_col in enumerate(sql_id_fat):
    fig.add_trace(go.Scatter(x=list(df_temp[df_temp.SQL_ID ==name_col]['BEGIN_INTERVAL_TIME']),
                  y=list(df_temp[df_temp.SQL_ID ==name_col]['mean']),
                  name=name_col, mode='lines+markers'
                  ),
                  int(i/rows+1),i%cols+1)
fig.update_layout(legend_orientation="v", hovermode="x",
                  margin=dict(l=30, r=30, t=60, b=30),
                  title='FATs SQL_IDs(ELA constant growth)')
fig.update_traces(hoverinfo="all", hovertemplate="<br>ELA: %{y}")
fig.show()

In [39]:
df_fats_simple.index

Index(['0b639nx4zdzxr', '42mkua34ay6bc', '63b54a3uuwp2h', '6qj2xst5028w4',
       'g6ax5u452xqqg', 'gnvpyyjvvnrtt', 'gq7yzd6cz83zt'],
      dtype='object', name='SQL_ID')

# CONCLUSION

*   Despite the stable load profile on the database (without subsidence and jumps), the execution time part of sql_id still degrades.
*   We found 8 sql_id, but it is worth taking 5 of them for optimization:
'0b639nx4zdzxr', '63b54a3uuwp2h', '6qj2xst5028w4','gnvpyyjvvnrtt', 'gq7yzd6cz83zt'